In [1]:
import numpy as np
from LinearRegression import OLS_Regression, LWLR_Regression, Ridge_Regression, FS_Regression
from sklearn.model_selection import train_test_split

In [11]:
## convert text file into matrix
def file2matrix(filename , header = True, delimeter="\t", index_y = None):
    """
    Takes a .txt file and returns a list of column names,a matrix of features and
    a vectore of y in case of having a target variable.
    """    
    fr = open(filename)
    colName = []
    if header == True:
        colName = list(fr.readline().strip().split(delimeter))
    numberOfLines = len(fr.readlines())
    fr = open(filename)
    numberOfX = len(fr.readline().split(delimeter)) 
    if index_y is not None:
        numberOfX = len(fr.readline().split(delimeter))-1                   
    returnMatX = np.zeros((numberOfLines,numberOfX))
    classLabelVector = []
    fr = open(filename)
    firstRow = 0
    if header == True:
        firstRow = 1
    index = 0
    for line in fr.readlines()[firstRow:]:
        line = line.strip()                            
        listFromLine = line.split(delimeter)
        if index_y is not None:
            classLabelVector.append(float(listFromLine[index_y]))
            listFromLine.pop(index_y)
        fltListFromLine = list(map(lambda x: float(x) if x!="" else np.nan, listFromLine))                                 
        returnMatX[index,:] = fltListFromLine        
        index += 1
    return colName, returnMatX, classLabelVector

In [12]:
## Accuracy measures 
def compute_error(trues,predicted):
    corr=np.corrcoef(predicted,trues)[0,1]
    mae=np.mean(np.abs(predicted-trues))
    rae=np.sum(np.abs(predicted-trues))/np.sum(np.abs(trues-np.mean(trues)))
    rmse=np.sqrt(np.mean((predicted-trues)**2))
    r2=max(0,1-np.sum((trues-predicted)**2)/np.sum((trues-np.mean(trues))**2))
    return corr,mae,rae,rmse,r2

In [13]:
filename = "abalone.txt"
colName, X, y = file2matrix(filename, header = False, index_y = -1)
y = np.array(y)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

**Ordinary Least Square (OLS) method:** 

$\hat w = (X^T . X)^{-1} . X^T . y$ 

In [15]:
model = OLS_Regression(fit_intercept=True, normalize=False, method="OLS")
model.fit(X_train, y_train)
pred = model.prediction(X_test)
corr, MAE, RAE, RMSE, R2 = compute_error(y_test,pred)
print("\nCorrCoef: %.3f\nMAE: %.3f\nRMSE: %.3f\nR2: %.3f" %(corr, MAE,RMSE, R2))


CorrCoef: 0.715
MAE: 1.607
RMSE: 2.221
R2: 0.511


In [6]:
## Using gradient descent
model = OLS_Regression(fit_intercept=True, normalize=False, eps=0.001, method="GD", alpha=0.1)
model.fit(X_train, y_train)
pred = model.prediction(X_test)
corr, MAE, RAE, RMSE, R2 = compute_error(y_test,pred)
print("\nCorrCoef: %.3f\nMAE: %.3f\nRMSE: %.3f\nR2: %.3f" %(corr, MAE,RMSE, R2))

iteration:   0 & loss value: 107.598
iteration: 100 & loss value: 6.694
iteration: 200 & loss value: 6.447
iteration: 300 & loss value: 6.236
iteration: 400 & loss value: 6.054
iteration: 500 & loss value: 5.898
iteration: 600 & loss value: 5.763
iteration: 700 & loss value: 5.647

CorrCoef: 0.660
MAE: 1.776
RMSE: 2.485
R2: 0.426


**Locally weighted linear regression (LWLR) method:**

We give a weight to data points near our data point
of interest

$\hat w = (X^T . WX)^{-1} . X^T . Wy$

In [16]:
model = LWLR_Regression(kernel="guassian", k=1)
pred = model.prediction(X_test, X_train, y_train)
corr, MAE, RAE, RMSE, R2 = compute_error(y_test,pred)
print("CorrCoef: %.3f\nMAE: %.3f\nRMSE: %.3f\nR2: %.3f" %(corr, MAE,RMSE, R2))

CorrCoef: 0.679
MAE: 1.622
RMSE: 2.366
R2: 0.433


**Ridge regression:**

$\hat w = (X^T . X + \lambda I)^{-1} . X^T . y$

Ridge regression was originally developed to deal with the problem of having more features than data points. But it can also be used to add bias into our estimations, giving us a better estimate. We can use the $\lambda$ value to impose a maximum value on the sum of all our $ws$. By imposing this penalty, we can decrease unimportant parameters.
This decreasing is known as $shrinkage$ in statistics.

**Importnat:** To use ridge regression and all shrinkage methods, you need to first normalize your features.

In ridge regression $\sum_{k=1}^n w_k^2 \leq \lambda$ meaning that the sum of the squares of all our weights has to be less than or equal to $\lambda$ . When two or more of the features are correlated, we may have a very large positive
weight and a very large negative weight using regular least-squares regression. By using ridge regression we’re avoiding this problem because the weights are subject to the previous constraint.

In [17]:
# Using OLS method for parameter estimation
model = Ridge_Regression(fit_intercept=True, normalize= True, method="OLS", lamb = 0.1)
model.fit(X_train, y_train)
print("Coeficients:", np.round(model.coef_,3))
pred = model.prediction(X_test)
corr, MAE, RAE, RMSE, R2 = compute_error(y_test,pred)
print("\nCorrCoef: %.3f\nMAE: %.3f\nRMSE: %.3f\nR2: %.3f" %(corr, MAE,RMSE, R2))

Coeficients: [ 0.021  0.024  0.348  0.135  1.469 -1.416 -0.339  0.343]

CorrCoef: 0.715
MAE: 1.614
RMSE: 2.224
R2: 0.509


In [23]:
# Using Stochastic gradient descent 
model = Ridge_Regression(fit_intercept=True, normalize= True, method="SGD", lamb = 0.1)
model.fit(X_train, y_train)
print("\nCoeficients:", np.round(model.coef_,3))
pred = model.prediction(X_test)
corr, MAE, RAE, RMSE, R2 = compute_error(y_test,pred)
print("\nCorrCoef: %.3f\nMAE: %.3f\nRMSE: %.3f\nR2: %.3f" %(corr, MAE,RMSE, R2))

iteration:   0 & loss value: 1.069
iteration: 100 & loss value: 0.762
iteration: 200 & loss value: 0.719
iteration: 300 & loss value: 0.707
iteration: 400 & loss value: 0.708
iteration: 500 & loss value: 0.716
iteration: 600 & loss value: 0.725
iteration: 700 & loss value: 0.734
iteration: 800 & loss value: 0.743
iteration: 900 & loss value: 0.750

Coeficients: [ 0.014  0.108  0.249  0.154  0.183 -0.771 -0.075  0.757]

CorrCoef: 0.705
MAE: 1.633
RMSE: 2.252
R2: 0.497


**Lasso regression**:

The lasso imposes a different constraint on the weights $\sum_{k=1}^n |w_k| \leq \lambda$. The only difference is that we’re taking the absolute value instead of the square of all the weights. To solve this we now need a quadratic programming algorithm. Instead of using the quadratic solver, we can use an easier method for getting results similar to the lasso. This is called **forward stagewise regression**.

#### Forward-stagewise algorithm:

In [ ]:
# Regularize the data to have 0 mean and unit variance
# For every iteration:
    # Set lowestError to +inf
        # For every feature:
            # For increasing and decreasing:
                # Change one coefficient to get a new W
                # Calculate the Error with new W
                # If the Error is lower than lowestError: set Wbest to the current W
            # Update set W to Wbest

In [8]:
model = FS_Regression()
model.fit(X_train, y_train)
print("Coeficients:", model.coef_)
pred = model.prediction(X_test)
corr, MAE, RAE, RMSE, R2 = compute_error(y_test,pred)
print("\nCorrCoef: %.3f\nMAE: %.3f\nRMSE: %.3f\nR2: %.3f" %(corr, MAE,RMSE, R2))

Coeficients: [ 0.01 -0.03  0.34  0.14  1.4  -1.39 -0.26  0.37]

CorrCoef: 0.729
MAE: 1.616
RMSE: 2.252
R2: 0.528
